<div style="text-align: left; background-color:#1ED760; font-family:Times New Roman; color:#191414; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 40px;border-style: solid;border-color: #1DB954;"><strong>ĐỒ ÁN MÔN HỌC</strong></div>
<div style="text-align: left; background-color:#FFFFFF; font-family: Times New Roman; color:black; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 18px"><strong>| CSC14119 – NHẬP MÔN KHOA HỌC DỮ LIỆU - NHÓM 16 |<strong></div>

## Bảng thành viên
|<center><div style="width:150px">MSSV</div><center>|<center><div style="width:290px">Họ và tên</div><center>|
|---------- |:-------------:|
| <center>21120112<center>  |<center>Bùi Kim Phúc<center> |

## Giới thiệu đề tài

- Trong những năm gần đây, ô nhiễm không khí đang trở thành một vấn đề môi trường nghiêm trọng trên toàn cầu. Theo báo cáo của Tổ chức Y tế Thế giới (WHO) năm 2022, ô nhiễm không khí là nguyên nhân gây tử vong cao thứ tư trên thế giới, chỉ đứng sau cao huyết áp, suy dinh dưỡng và hút thuốc lá. Ô nhiễm không khí gây ra khoảng 7 triệu ca tử vong sớm mỗi năm, trong đó 6,6 triệu ca tử vong xảy ra ở các nước đang phát triển.

- Tình trạng ô nhiễm không khí ở Việt Nam cũng đang ở mức báo động. Theo báo cáo của Bộ Tài nguyên và Môi trường Việt Nam năm 2022, 80% các thành phố lớn ở Việt Nam có chất lượng không khí không đáp ứng các tiêu chuẩn của WHO.

*-> Vì vậy phân tích chất lượng không khí là một công cụ quan trọng để theo dõi xu hướng ô nhiễm không khí, xác định các nguồn gây ô nhiễm và đánh giá tác động của ô nhiễm không khí đối với sức khỏe và môi trường. Đó cũng là lí do nhóm quyết định thu thập dữ liệu về chất lượng không khí ở các thành phố lớn, cụ thể là thành phố Hồ Chí Minh, để có những cái nhìn về chất lượng không khí nơi đó cũng như tại các thành phố lớn trên thế giới.*

## Import các thư viện cần thiết

In [1]:
import requests
import json
import pandas as pd
import time
import datetime
import calendar
import seaborn as sns
import matplotlib.pyplot as plt

## Thu thập dữ liệu

- Chúng em sẽ thu thập dữ liệu thông qua api do trang https://openweathermap.org đã cung cấp sẵn và lưu và biến data.
    

- Mãu Api: http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={APIkey}
<table class="material-table">
                  <tr>
            <th colspan="3">Parameters</th>
                  </tr>
               <tr>
                     <td><code class="nowrap">lat</code></td>
                     <td class="sub">required</td>
                     <td>Latitude. If you need the geocoder to automatic convert city names and zip-codes to geo coordinates and the other way around, please use our <a href="/api/geocoding-api">Geocoding API</a></td>
                  </tr>
                     <tr>
                     <td><code class="nowrap">lon</code></td>
                     <td class="sub">required</td>
                     <td>Longitude. If you need the geocoder to automatic convert city names and zip-codes to geo coordinates and the other way around, please use our <a href="/api/geocoding-api">Geocoding API</a></td>
                  </tr>
                  <tr>
                     <td><code class="nowrap">start</code></td>
                     <td class="sub">required</td>
                     <td>Start date (unix time, UTC time zone), e.g. start=1606488670</td>
                  </tr>
                  <tr>
                     <td><code class="nowrap">end</code></td>
                     <td class="sub">required</td>
                     <td>End date (unix time, UTC time zone), e.g. end=1606747870</td>
                  </tr>
                  <tr>
                     <td><code class="nowrap">appid</code></td>
                     <td class="sub">required</td>
                     <td>Your unique API key (you can always find it on your account page under the <a href="https://home.openweathermap.org/api_keys" target="_blank">"API key" tab</a>)</td>
                  </tr>
               </table>

Api yêu cầu ngày tháng phải ở dạng Unix Time, chúng em sử dụng hàm bên dưới để chuyển từ định dạng datetime thành Unix time. Nguồn tham khảo: https://stackoverflow.com/questions/1077285/how-to-specify-time-zone-utc-when-converting-to-unix-time-python

In [2]:
def date_time_to_utc_epoch(dt_utc):         #convert from utc date time object (yyyy-mm-dd hh:mm:ss) to UTC epoch
    frmt="%Y-%m-%d %H:%M:%S"
    dtst=dt_utc.strftime(frmt)              #convert datetime object to string
    time_struct = time.strptime(dtst, frmt) #convert time (yyyy-mm-dd hh:mm:ss) to time tuple
    epoch_utc=calendar.timegm(time_struct)  #convert time to to epoch
    return str(epoch_utc)

Thiết kế hàm để load data từ api được cung cấp

In [3]:
# Base url của API
base_url = 'http://api.openweathermap.org/data/2.5/air_pollution/history?'

# Hàm load data
def load_data(lat: str, lon: str, start_date: str, end_date: str, appid: str):
    """Hàm lấy dữ liệu

    Args:
        lat (str): Vĩ độ.
        lon (str): Kinh độ.
        start_date (str): Ngày bắt đầu (thời gian unix, múi giờ UTC).
        end_date (str): Ngày kết thúc (thời gian unix, múi giờ UTC).
        appid(str): Khóa API

    Returns:
        pd.Dataframe: Bảng dữ liệu
    """
    params = {
        'lat': lat,
        'lon': lon,
        'start': start_date,
        'end': end_date,
        'appid': appid
    }
    r = requests.get(base_url, params=params)
    air_quality_df = pd.DataFrame()
    if r.status_code == 200:
        data = json.loads(r.text)
        extracted_data = [{'dt': item['dt'], 'aqi': item['main']['aqi'], **item['components']} for item in data['list']]
        air_quality_df = pd.DataFrame(extracted_data)
    return air_quality_df

Tiến hành load data từ API. Chúng em sẽ tiến hành lấy dữ liệu của *TP Hồ Chí Minh từ 01/01/2021 tới 30/11/2024*

In [4]:
# Định nghĩa params
start_date = date_time_to_utc_epoch(datetime.datetime(2021,1,1)) # Chuyển ngày 01/01/2021 về dạng Unix Time dựa vào hàm chuyển đổi
end_date = date_time_to_utc_epoch(datetime.datetime(2024,11,30)) # Chuyển ngày 30/11/2024 về dạng Unix Time dựa vào hàm chuyển đổi

# Tọa độ của TP HCM
lat = '10.8231'
lon =  '106.6297'

# Key id được tạo cung cấp từ trang web
appid =  "59da78cf88e29ed9966a7904d10dbf34"

# Load data
df = load_data(lat, lon, start_date, end_date, appid)
df

,dt,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,1609459200,3,700.95,0.44,35.99,17.35,32.90,20.33,26.64,8.99
1,1609462800,3,847.82,2.46,38.04,18.06,36.24,23.32,30.54,9.37
2,1609466400,3,894.55,5.25,38.39,23.25,41.01,24.16,31.93,9.25
3,1609470000,3,827.79,6.20,36.33,33.98,43.39,23.20,30.91,8.61
4,1609473600,2,660.90,3.69,29.13,54.36,35.76,19.50,25.60,6.21
...,...,...,...,...,...,...,...,...,...,...
33812,1732910400,2,600.81,1.30,37.70,5.99,23.13,21.54,27.61,9.25
33813,1732914000,2,554.08,0.75,35.99,8.85,23.13,20.50,26.39,8.36
33814,1732917600,2,567.44,0.64,36.67,10.19,24.80,22.20,28.90,8.04
33815,1732921200,2,600.81,0.76,37.36,9.66,26.23,24.03,32.19,8.61


Lưu data dưới dạng csv

In [5]:
df.to_csv('../Data/air_quality.csv', sep=',', encoding='utf-8', index=False)